## Digital Signature Schemes over the Ring of Gaussian Integers


In [1]:
import numpy as np
import struct

# Imaginary number i
img = complex(0+1j)

In [2]:
def mod_complex(z, m):  
    """
    Calculates the a+bi(modk).
    z: Complex value
    m: Module
    """
    
    x = np.around((z.real/m)%1*m-m)
    y = np.around((z.imag/m)%1*m-m)
    r = x + y*img
    return r
mod_complex(3+4j, 4)

(-1-4j)

In [3]:
def split_mod(b, s, m):
    """
    Calculates the (a+bi)^x(modek).
    b: Complex value
    s: Upper
    m: Module
    """
    
    sseed = list()
    lseed = list()
    
    k = int(np.log2(s))
    while s>0:
        v = int(np.log2(s))
        lseed.append(v)
        s = s-np.power(2, lseed[-1])
    
    seed = b
    sseed.append(seed)
    
    for i in range(k):
        seed = mod_complex(np.power(seed, 2), m)
        sseed.append(seed)
 
    z = 1
    for seed in lseed:
        z *= sseed[seed]
    z = mod_complex(z, m)
    return z
print(split_mod(3+2j,4,4))

(-3-4j)


In [4]:
def calc_mod_equal_one(b, m, v):
    """
    Calculates the b*h=v(mod m).
    
    Note:
    -----------------------------
    This method uses quick scan up to module+1. 
    It may not perform if b*h>2^^64.
    """
    
    for i in range(int(m)+1):
        if b*i%m == v:
            return i

In [5]:
def N(comp_val):
    """
    N function which translates complex(a + bi) to (a^2 + b^2).
    """
    
    if not isinstance(comp_val, complex):
        print("Warn: N is not complex value. Making it complex.")
        comp_val = complex(comp_val + 0j)
    return int(np.power(comp_val.real, 2) + np.power(comp_val.imag, 2))


## Prepare
Alice wants to sing a message.

Suppose H(m)=2022 as below.

In [6]:
# Alice message and hashed message. 
hashed_message = 2022
print("Alice's Hashed Message: ", hashed_message)

Alice's Hashed Message:  2022


## Alice Step 1
Alice chooses gaussian primes, alfa and calculates the Phi(alfa).

In [7]:
gaussian_prime_1 = 11
gaussian_prime_2 = 19
gaussian_primes = (gaussian_prime_1, gaussian_prime_2)

alfa = gaussian_prime_1*gaussian_prime_2
print("Alice chooses Gaussian Primes: ", gaussian_primes)
print("Calculated Alfa: ", alfa)

phi_alfa = (N(complex(gaussian_prime_1+0j))-1) * (N(complex(gaussian_prime_2+0j))-1)
print("Phi(alfa): ", phi_alfa)

Alice chooses Gaussian Primes:  (11, 19)
Calculated Alfa:  209
Phi(alfa):  43200


## Alice Step 2
Alice chooses a gaussian integer Beta and a random integer A.  

In [8]:
gaussian_comp = np.complex256(7 + 13j)
random_positive_integer = 331
z = split_mod(gaussian_comp, random_positive_integer, alfa)
print("Alice chooses Gaussian Integer Beta: ", gaussian_comp)
print("Alice chooses a random positive integer A: ", random_positive_integer)
print("Module of a-Raised-Beta: ", z)

Alice chooses Gaussian Integer Beta:  (7+13j)
Alice chooses a random positive integer A:  331
Module of a-Raised-Beta:  (-125-53j)


## Alice Step 3
Alice selects an encryption exponent e.

In [9]:
e = 1391
print("Alice chooses encryption exponent: ", e)
eta = split_mod(gaussian_comp, e, alfa)

print("Module of e-Raised-Beta: ", eta)

Alice chooses encryption exponent:  1391
Module of e-Raised-Beta:  (-92-46j)


## Alice Step 4
Alice finds the unique integer h value.

In [10]:
h = calc_mod_equal_one(e, phi_alfa, 1)
print("Unique integer h: ", h)

Unique integer h:  15311


## Alice Step 5
Alice calculates s value and assembles his public key and signed message.

In [11]:
s = h*(hashed_message-random_positive_integer)%phi_alfa
print("Alice computes s: ", s)

alice_public_key = (alfa, gaussian_comp, z)
signed_message = (hashed_message, s, eta)
print("Alice public key: ", alice_public_key)
print("Signed message: ", signed_message)

Alice computes s:  14101
Alice public key:  (209, (7+13j), (-125-53j))
Signed message:  (2022, 14101, (-92-46j))


# Bob Step 1 and 2
Suppose Bob downloads the Alice's public key.

With using downloaded key Bob computes following.

In [12]:
c = split_mod(eta, s, alfa)
verify_val = mod_complex(z*c, alfa) 

# Eta and s values from signed message
# alfa, z from Alice's pubic key.   
print("Bob calculates: ", c)
print("Bob calculates verify value: ", verify_val)

Bob calculates:  (-81-145j)
Bob calculates verify value:  (-68-154j)


## Bob Step 3
Bob checks the signature.

In [13]:
mod_of_hash_raised_beta = split_mod(gaussian_comp, hashed_message, alfa)

if mod_of_hash_raised_beta == verify_val:
    print(f"Bob verifies the signature as {mod_of_hash_raised_beta} equals {verify_val}.")
else:
    print(f"Bob cannot verify the signature as {mod_of_hash_raised_beta} not equals {verify_val}.")

Bob verifies the signature as (-68-154j) equals (-68-154j).
